# Pobieranie danych

Download zip file from: http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip \
Unpack it and put the files do /IDS2017/Raw folder

# Importy i ważne zmienne


In [1]:
import numpy as np
import pandas as pd
import os
from fastai.tabular.all import df_shrink

# Czytanie ścieżek do plików i danych

In [2]:
def read_paths(dir_path):
  dspaths = []
  for dirname, _, filenames in os.walk(dir_path):
      for filename in filenames:
          if filename.endswith('.csv'):
              pds = os.path.join(dirname, filename)
              dspaths.append(pds)
              print(pds)
  return dspaths

def read_files(dspaths):
  individual_dfs = [pd.read_csv(dsp, sep=',', encoding='utf-8') for dsp in dspaths]
  [i.shape for i in individual_dfs]
  return individual_dfs

dspaths = read_paths('./IDS2017/Raw')
individual_dfs = read_files(dspaths)

./IDS2017/Raw\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
./IDS2017/Raw\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
./IDS2017/Raw\Friday-WorkingHours-Morning.pcap_ISCX.csv
./IDS2017/Raw\Monday-WorkingHours.pcap_ISCX.csv
./IDS2017/Raw\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
./IDS2017/Raw\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
./IDS2017/Raw\Tuesday-WorkingHours.pcap_ISCX.csv
./IDS2017/Raw\Wednesday-workingHours.pcap_ISCX.csv


In [3]:
individual_dfs[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286467 entries, 0 to 286466
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             286467 non-null  int64  
 1    Flow Duration                286467 non-null  int64  
 2    Total Fwd Packets            286467 non-null  int64  
 3    Total Backward Packets       286467 non-null  int64  
 4   Total Length of Fwd Packets   286467 non-null  int64  
 5    Total Length of Bwd Packets  286467 non-null  int64  
 6    Fwd Packet Length Max        286467 non-null  int64  
 7    Fwd Packet Length Min        286467 non-null  int64  
 8    Fwd Packet Length Mean       286467 non-null  float64
 9    Fwd Packet Length Std        286467 non-null  float64
 10  Bwd Packet Length Max         286467 non-null  int64  
 11   Bwd Packet Length Min        286467 non-null  int64  
 12   Bwd Packet Length Mean       286467 non-nul

In [4]:
individual_dfs[1].sample()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
161586,5904,63,1,1,0,6,0,0,0.0,0.0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,PortScan


In [5]:
print(individual_dfs[1].isnull().sum())
print(individual_dfs[1].notna().sum())

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64
 Destination Port              286467
 Flow Duration                 286467
 Total Fwd Packets             286467
 Total Backward Packets        286467
Total Length of Fwd Packets    286467
                                ...  
Idle Mean                      286467
 Idle Std                      286467
 Idle Max                      286467
 Idle Min                      286467
 Label                         286467
Length: 79, dtype: int64


# Downsizing

In [6]:
individual_dfs[0].dtypes

 Destination Port                int64
 Flow Duration                   int64
 Total Fwd Packets               int64
 Total Backward Packets          int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
 Idle Std                      float64
 Idle Max                        int64
 Idle Min                        int64
 Label                          object
Length: 79, dtype: object

In [7]:
individual_dfs = [df_shrink(df) for df in individual_dfs]

In [8]:
individual_dfs[0].dtypes

 Destination Port                 int32
 Flow Duration                    int32
 Total Fwd Packets                int16
 Total Backward Packets           int16
Total Length of Fwd Packets       int32
                                 ...   
Idle Mean                       float32
 Idle Std                       float32
 Idle Max                         int32
 Idle Min                         int32
 Label                         category
Length: 79, dtype: object

# Usuwanie niezdefiniowanych wartości

In [9]:
def drop_nan(individual_dfs):
  for df in individual_dfs:
      df.replace([np.inf, -np.inf], np.nan, inplace=True)
      # print(df.isna().any(axis=1).sum(), "rows with at least one NaN to remove")
      df.dropna(inplace=True)
  [i.shape for i in individual_dfs]
  return individual_dfs

individual_dfs = drop_nan(individual_dfs)

# Usuwanie zduplikowanych wierszy

In [10]:
def drop_dupes(individual_dfs):
  for df in individual_dfs:
      print(df.duplicated().sum(), "fully duplicate rows to remove")
      df.drop_duplicates(inplace=True)
      df.reset_index(inplace=True, drop=True)
  [i.shape for i in individual_dfs]
  return individual_dfs

individual_dfs = drop_dupes(individual_dfs)

2629 fully duplicate rows to remove
72319 fully duplicate rows to remove
6867 fully duplicate rows to remove
26831 fully duplicate rows to remove
35605 fully duplicate rows to remove
6052 fully duplicate rows to remove
24019 fully duplicate rows to remove
80914 fully duplicate rows to remove


## Zapisywanie danych w CSV

In [9]:
def convert(individual_dfs):
  for i, df in enumerate(individual_dfs):
        csv_path = f"./IDS2017/Clean/{dspaths[i].split('/')[-1].replace('.pcap_ISCX', '')[4:]}"
        df.to_csv(csv_path, index=False)

convert(individual_dfs)

## Zapisywanie połączonych danych w CSV

In [ ]:
combined_df = pd.concat(individual_dfs, ignore_index=True)
combined_df.to_csv("combined.csv", index=False)


# Zapisywanie danych w parquet



In [11]:
def convert(individual_dfs):
  for i, df in enumerate(individual_dfs):
      df.to_parquet(f"./IDS2017/Clean/{dspaths[i].split('/')[-1].replace('.pcap_ISCX.csv', '.parquet')[4:]}")

convert(individual_dfs)

## Zapisywanie połączonych danych w parquet

In [ ]:
def convert_to_single_parquet(individual_dfs, output_path):
    combined_df = pd.concat(individual_dfs, ignore_index=True)
    combined_df.to_parquet(output_path)

convert_to_single_parquet(individual_dfs, "combined.parquet")

# Czytanie plików i misc

## Zamienianie wartośći z kolumny Label na BENINGN/ATTACK


In [ ]:
label_map = {'BENIGN': 'BENIGN', 'DDoS': 'ATTACK', 'DoS Hulk': 'ATTACK', 'DoS GoldenEye': 'ATTACK', 'DoS slowloris': 'ATTACK',
             'FTP-Patator': 'ATTACK', 'SSH-Patator':  'ATTACK', 'DoS Slowhttptest': 'ATTACK', 'PortScan': 'ATTACK',
             'Web Attack � Brute Force' : 'ATTACK', 'Bot': 'ATTACK', 'Web Attack � XSS': 'ATTACK', 'Infiltration': 'ATTACK',
             'Web Attack � Sql Injection': 'ATTACK', 'Heartbleed': 'ATTACK'}
combined01['Label'] = combined01['Label'].map(label_map).fillna('ATTACK')

print(combined01['Label'].value_counts())

Label
BENIGN    1886548
ATTACK     316445
Name: count, dtype: int64
